In [1]:
parkruns = pd.read_csv("parkruns.csv")

NameError: name 'pd' is not defined

In [ ]:
chunk_size = 10
parkruns.dropna(subset=["Time"], inplace=True)
parkruns["ChunkNum"] = parkruns["ParkrunNum"]//chunk_size

In [ ]:
def parse(s):
    if len(s.split(":")) == 2:
        return f"00:{s}"
    return s
parkruns = parkruns[parkruns["parkrunner"]!="Unknown"]
parkruns["Time"] = parkruns["Time"].apply(parse)
parkruns["Time"] = pd.to_timedelta(parkruns["Time"])

In [ ]:
best_in_chunk = parkruns.loc[parkruns.groupby(["ChunkNum", "parkrunner"])["Time"].idxmin()]

In [ ]:
top_10_in_chunk = best_in_chunk.groupby("ChunkNum").apply(lambda x: x.sort_values("Time")[:10])
top_10_in_chunk["WRTimeRel"] = top_10_in_chunk["Time"] - pd.Timedelta(minutes=16, seconds=48)
#top_10_in_rolling = parkruns.rolling(5, on="ParkrunNum")["Time"].apply(lambda x: sorted(x, ascending=False)[:10])

In [ ]:

def bar_for_chunk(chunk):
    s = top_10_in_chunk[top_10_in_chunk["ChunkNum"]==chunk].sort_values("WRTimeRel", ascending=False)
    marker_colors = [f"#{hex(hash(str(i)))[-6:]}"  for i in s["parkrunner"]] 
    y= s["WRTimeRel"].apply(lambda x: x.seconds)
    return  y, s["parkrunner"], marker_colors





In [ ]:
import time
x, y, m = bar_for_chunk(0)
fig = go.Figure([go.Bar(y=y, x=x, orientation='h', marker_color=m)])
fig

In [ ]:
frames = []
for i in range(1,top_10_in_chunk["ChunkNum"].max()):
    x, y, m = bar_for_chunk(i)
    dat = go.Bar(y=y, x=x, orientation='h', marker_color=m)
    frames.append({"data":dat, "name":f"{i}"})
fig["frames"] = frames

In [ ]:
from plotly.offline import init_notebook_mode, iplot
iplot(fig, "parkrun")